## Estrategia

Calcular en qué unidades electorales es más fácil sacar votos, a la vez que podemos asignar quién sería el beneficiario de esa ganancia. Esto permite asignar visitas y mensajes a cada candidato. Estos mensajes pueden ser de pedir cambiar el voto, y de conseguir que los no votantes voten. La facilidad de cambiar el voto es proporcional a cómo de saturada esté la votación.


## Hipótesis

1. Los resultados de las votaciones son iguales a las anteriores, o extrapolaciones de la tendencia.
2. El *efecto del esfuerzo* para ganar una % de votos satura en ambos extremos, es una gaussiana. En los extremos (cuando casi nadie o casi todos ya votan) cuesta mucho ganar votos, en el centro es más fácil.
3. Los votos ganados tienen que salir de uno de estos dos sitios: a)de votos a otros partidos, cuya resistencia a cambiar es la misma gausiana que en 2. b) de motivar a los no votantes, que votarán en primera approximación proporcionalmente a predición de resultados. I.e. la estrategia puede ser "ganar votos para un partido" o "id a votar".
4. El resultado del esfuerzo es proporcional al tamaño de la unidad de voto. Aunque el esfuerzo sea sobre el % de votos de una divisón electoral, hay divisiones con 100 candidatos, y otras con cientos de miles. 
5. Al ser España una circumscripción única para las Europeas, de listas cerradas, se puede asignar nombre a los votos ganados o perdidos en cada actuación. E.g. 10 votos más sacan al candidato X.
* Por tanto, los efectos de los esfuerzos de voto se pueden asiganr a los candidatos correspodientes en la asignación de escaños. E.g. En el minicipio X se prevé que el esfuerzo es mínimo para sacar el escaño, y el mensaje es simplemente que voten.

Por tanto, la idea es que, para cada escaño, calcular cuantos votos hacen falta para asignárselo. Luego ordenar los lugares por orden del resultado del esfuerzo. Uno podría minimizarlo con el coste del esfuerzo, pero en principio se ordena por el efecto del esfuerzo


Dado un %% de votos, se asigna un tiron  Tx=gauss(x). Este tirón se tiene que compensar con una serie de tirones de Ta,Tb,Gc, ... y N. Siendo Ta=agauss(A) lo tirones negativos hasta que se consigue Tx. Tn es el tirón de los no votantes. El tirón de los no votantes tiene el effecto proporcional a los % de votos, por lo tanto afecta a todos. 


In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np

def sigmoid(x):
    stdev=10; mean=50;
    return 100/(1+math.exp(-(x-mean)/stdev))

#sigmoid function
x = np.arange(0., 100., 0.2)
sig=[]
for item in x:
    sig.append(sigmoid(item))
    
#resultados electorales
resultados={
    'NO':40,
    'A': 55,
    'B': 30,
    'C': 10,
    'D': 5
}
    
    
plt.plot(x,sig)
plt.xlabel('% de votos totales')
plt.ylabel('% de votos a candidatura')

for key in resultados.keys():
    plt.text(resultados[key],sigmoid(resultados[key]),'x '+key)
plt.show()


## Datos

* Sacar los datos de elecceiones de 2014 granulares de los municipios
* Buscar los sitios de menor participación.

* Sacar las diferencias de participación.


Busque en Wikipedia, en github, en el boe, pero nadie tiene los CSV de resultados desagregados.

Al final encontré la mayoría de datos en http://elecciones.mir.es/resultados2014/ini99v.htm

Auqneu son provisionales al 99.77% me valen.
No tienen opcion a descargas todo, ni va con API que pueda ver. Pero si tienen un ejecutable con todos los datos. Al descargarlo (en wine con OSX) veo que tiene una base de datos que quizás sí tenga todos los datos desagregados. Es un .mdb (base de dato de Microsoft Office) de 132MB.

Usé esta página para exportar los datos
https://www.codeenigma.com/community/blog/using-mdbtools-nix-convert-microsoft-access-mysql

```sql
mdb-schema Europeas2014.mdb mysql > schema.sql
mkdir sql
for i in $( mdb-tables Europeas2014.mdb ); do echo $i ; mdb-export -D "%Y-%m-%d %H:%M:%S" -H -I mysql Europeas2014.mdb $i > sql/$i.sql; done
mysql -uroot -ppassword europeas < schema.sql
for i in $( ls sql/ ); do echo $i ; mysql -uroot -ppassword europeas < sql/$i ; done
```
Dará error al importar el sql Partidos, pero es un fichero raro binario, ni idea de lo que es.

Si miras la tabla "resultados" verás que pone:


In [ ]:
#Connect to de DB
import os
import pymysql
import pandas as pd

host = os.getenv('MYSQL_HOST')
port = os.getenv('MYSQL_PORT')
user = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')
database = 'europeas'

conn = pymysql.connect(
    host=host,
    port=int(port),
    user=user,
    passwd=password,
    db=database,
    charset='utf8mb4')

In [ ]:
query="select * from resultados limit 1"
df = pd.read_sql_query(query,conn)
df.to_dict('index')




Esto quiere decir que la información de los nombres está codificada. Por ejemplo para saber qué distrito es `CANDIDATURA: 0002`


In [ ]:
query="select * from candidaturas where candidatura=0002;"
df = pd.read_sql_query(query,conn)
df.to_dict('index')


## Resultados por municipio

Como los datos están consignados, si quiero saber lo que votó en mi pueblo "Soto del Barco", primero tengo que sacar su consigna:

In [ ]:
query="select * from municipios where nombre='Soto del Barco'"
df = pd.read_sql_query(query,conn)
df.to_dict('index')


In [ ]:
query="select * from municipios where AUTONOMIA=03 and PROVINCIA=33 and MUNICIPIO=069"
df = pd.read_sql_query(query,conn)
df

Ahora puedo pedir los resultados donde se cumplen esos criterios:

In [ ]:
query="""select tramo,candidatura,votos_act 
         from resultados 
         where AUTONOMIA=03 and PROVINCIA=33 and MUNICIPIO=069
         and votos_act>0
         order by votos_act desc;"""
df = pd.read_sql_query(query,conn)
df.to_dict('index')



y para ver esos nombres:

In [ ]:
query="""select c.sigla,votos_act,PVOTOS_ACT 
         from resultados as r 
         join candidaturas as c on r.candidatura=c.candidatura 
         where AUTONOMIA=03 and PROVINCIA=33 and MUNICIPIO=069
         and votos_act>0
         order by votos_act desc;"""
df = pd.read_sql_query(query,conn)
df.to_dict('index')


Para ver que vamos bien podemos mirar los resultados [en la web](http://elecciones.mir.es/resultados2014/99PE/DPE0333906999.htm?d=533), dentro del app:

![Soto del Barco](sotodelbarco.png)



# Histograma de % de votos por partido y Municipio

El modelo se basa en comparar el % de votos en cada unidad, con el conseguido en todas las unidades.

In [ ]:
query="""select c.sigla,r.PVOTOS_ACT 
         from resultados as r 
         join candidaturas as c on r.candidatura=c.candidatura 
         and r.PVOTOS_ACT >0
         order by c.sigla asc;"""
todos_votos = pd.read_sql_query(query,conn)
todos_votos.head(10)

In [ ]:
todos_votos.set_index(keys=['sigla'],inplace=True)

In [ ]:
todos_votos.hist(by='sigla',figsize=(20,20),sharex=True,sharey=False,density=1);

In [ ]:
query="select MESAS_ACT,CENSO_ACT,VOTOS_ACT,PVOTOS_ACT,VBLAN_ACT,PVBLAN_ACT,VNULOS_ACT,PVNULOS_ACT "+\
    "from escrutinio as e "+\
    "where"+\
    " e.CENSO_ACT>0;"
no_votantes = pd.read_sql_query(query,conn)
len(no_votantes)

In [ ]:
no_votantes.hist(column=['PVOTOS_ACT','PVBLAN_ACT','PVNULOS_ACT'],figsize=(10,10),bins=100,density=1);

---

# Ejemplo con un sitio

In [ ]:
sitio='Gijon'

In [ ]:
#get AUTONOMIA,PROVINCIA,MUNICIPIO
query="select * from municipios where nombre='"+sitio+"'"
df = pd.read_sql_query(query,conn)
lugar=df.to_dict('index')
if (len(lugar.keys())>1):
    print(lugar)
    raise RuntimeError('Más de un lugar con ese nombre!')
else:
    lugar=lugar[0]         
#print(lugar)

#Check if it corresponds to several tramos
query="select * from municipios where "+\
      "AUTONOMIA="+lugar['AUTONOMIA']+\
      " and PROVINCIA="+lugar['PROVINCIA']+\
      " and MUNICIPIO="+lugar['MUNICIPIO']
ddf = pd.read_sql_query(query,conn).to_dict('index')
if (len(ddf.keys())>1):
    print(ddf)
    raise RuntimeError('Más de un lugar en esa combinación de AUTONOMIA,PROVINCIA,MUNICIPIO!')

#Comprobar si en los resultados hay varios tramos con esa combinacion de AUTONOMIA,PROVINCIA,MUNICIPIO
query="select distinct tramo from resultados where "+\
      "AUTONOMIA="+lugar['AUTONOMIA']+\
      " and PROVINCIA="+lugar['PROVINCIA']+\
      " and MUNICIPIO="+lugar['MUNICIPIO']
df = pd.read_sql_query(query,conn)
ddf= df.to_dict('index')
if (len(ddf.keys())>1):
    print(ddf)
    raise RuntimeError('Más de un lugar de votaciones con en esa combinación de AUTONOMIA,PROVINCIA,MUNICIPIO!')

In [ ]:
query="select c.sigla,r.VOTOS_ACT,r.PVOTOS_ACT "+\
    "from resultados as r "+\
    "join candidaturas as c on r.candidatura=c.candidatura "+\
    "where r.PVOTOS_ACT >0"+\
    " and r.AUTONOMIA="+lugar['AUTONOMIA']+\
    " and r.PROVINCIA="+lugar['PROVINCIA']+\
    " and r.MUNICIPIO="+lugar['MUNICIPIO']+\
    " order by votos_act desc;"""
votos_lugar = pd.read_sql_query(query,conn)
votos_lugar.to_dict('index')

In [ ]:
query="select MESAS_ACT,CENSO_ACT,VOTOS_ACT,PVOTOS_ACT,VBLAN_ACT,PVBLAN_ACT,VNULOS_ACT,PVNULOS_ACT "+\
    "from escrutinio as e "+\
    "where"+\
    " e.AUTONOMIA="+lugar['AUTONOMIA']+\
    " and e.PROVINCIA="+lugar['PROVINCIA']+\
    " and e.MUNICIPIO="+lugar['MUNICIPIO']+\
    ";"
escrutinio_lugar = pd.read_sql_query(query,conn)
escrutinio_lugar.to_dict('index')

In [ ]:
#partidos_con_votos
query="select distinct c.sigla "+\
    "from candidaturas as c "+\
    "join resultados as r on r.candidatura=c.candidatura "+\
    "where r.VOTOS_ACT >0"+\
    " and r.AUTONOMIA="+lugar['AUTONOMIA']+\
    " and r.PROVINCIA="+lugar['PROVINCIA']+\
    " and r.MUNICIPIO="+lugar['MUNICIPIO']+\
    ";"""
partidos_con_votos = pd.read_sql_query(query,conn).values
partidos_con_votos



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

cols=4
rows=len(partidos_con_votos)/4 +1

plt.figure(figsize=(30,30))

  
k=0
#each party
for partido in partidos_con_votos:
    k=k+1
    partido=partido[0]
    todos_partido=todos_votos[todos_votos['sigla']==partido]['PVOTOS_ACT'].values
    este_partido=votos_lugar[votos_lugar['sigla']==partido]['PVOTOS_ACT'].values
    votos=votos_lugar[votos_lugar['sigla']==partido]['VOTOS_ACT'].values

    # the histogram of the data
    plt.subplot(rows,cols,k)
    
    n, bins, patches = plt.hist(todos_partido, 100, density=True, facecolor='g', alpha=0.75)
    plt.axvline(x=este_partido,linewidth=5)

    plt.xlim(np.percentile(todos_partido,1), np.percentile(todos_partido,99))
    plt.title(partido+' '+str(votos[0]))
    plt.grid(True)

#non voters
k=k+1
plt.subplot(rows,cols,k)

todos_no_votos=no_votantes['PVOTOS_ACT'].values
este_no_votos=escrutinio_lugar['PVOTOS_ACT'].values

n, bins, patches = plt.hist(todos_no_votos, 100, density=True, facecolor='b', alpha=0.75)
plt.axvline(x=este_no_votos,linewidth=5)

plt.xlim(np.percentile(todos_no_votos,1), np.percentile(todos_no_votos,99))
plt.title('NO VOTANTES '+str(escrutinio_lugar['VOTOS_ACT'].values))
plt.grid(True)

plt.show()

In [ ]:
partido='VOX'

todos_partido=todos_votos[todos_votos['sigla']==partido]['PVOTOS_ACT'].values
este_partido=votos_lugar[votos_lugar['sigla']==partido]['PVOTOS_ACT'].values

# the histogram of the data
n, bins, patches = plt.hist(todos_partido, 100, density=True, facecolor='g', alpha=0.75)
plt.axvline(x=este_partido,linewidth=5)

plt.axis([np.percentile(todos_partido,1), np.percentile(todos_partido,99),0,.8])
plt.xlabel('Distribución resultados nacional')
plt.ylabel('Probabilidad')
plt.title(partido)

plt.grid(True)
plt.rcParams["figure.figsize"] = [10,10]
plt.show()





## GEOLOCALIZACIÓN

Los datos geograficos salen de
http://centrodedescargas.cnig.es/CentroDescargas/index.jsp

También en formato mdb.
mdb-schema --namespace geo Nomenclator_Municipios_EntidadesDePoblacion.mdb mysql > schema.sql
mkdir sql
for i in $( mdb-tables Nomenclator_Municipios_EntidadesDePoblacion.mdb ); do echo $i ; mdb-export -D "%Y-%m-%d %H:%M:%S" -H -I mysql -N geo Nomenclator_Municipios_EntidadesDePoblacion.mdb $i > sql/$i.sql; done
mysql -uroot -ppassword europeas < schema.sql
for i in $( ls sql/ ); do echo $i ; mysql -uroot -ppassword europeas < sql/$i ; done

Hay dos entidades de lugar que son siempre 0:

```sql
select distinct COMARCA,TRAMO from resultados;
+---------+-------+
| COMARCA | TRAMO |
+---------+-------+
| 00      | 0     |
+---------+-------+
1 row in set (0.51 sec)
``


In [ ]:
conn.close()